In [113]:
# load dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [114]:
# Load data and select trainging hour and test hour
trainhour = 5
#testhour = 7
df = pd.read_csv('combined.csv')
cols = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
cols10 = ['31', '32', '33', '34', '35', '36', '37', '38', '39', '40']
df['begin'] = df[cols].sum(axis=1)
df['sum10'] = df[cols10].sum(axis=1)
data = df[(df['hour'] == trainhour) & (df['month'] == 2)]
#testdata = df[(df['hour'] == testhour) & (df['month'] == 6)]
del df

In [115]:
# Categorize the 'sum' column the number of PIPs over selected range
def setlabels(x):
    if x < -15:
        return -1
    elif x > 15:
        return 1
    else:
        return 0

data["labels"] = data["sum10"].apply(setlabels)

In [116]:
X_data = data.iloc[:,18:38]
X_data['begin'] = data['begin']
X_data['dow_open'] = data['dow_open']
X_data['euf_open'] = data['euf_open']
X_data['us_6mn'] = data['us_6mn']
X = X_data.to_numpy()
X[0:2]

array([[ 2.5       , -0.4       ,  2.6       , -1.2       ,  2.5       ,
         0.5       , -2.2       , -3.        ,  0.3       ,  2.        ,
         1.        , -0.9       , -1.2       , -1.7       ,  1.5       ,
        -2.1       , -3.6       ,  1.2       , -1.2       ,  1.8       ,
        -2.4       ,  2.60830391,  2.5299999 ,  1.64      ],
       [-0.4       ,  2.6       , -1.2       ,  2.5       ,  0.5       ,
        -2.2       , -3.        ,  0.3       ,  2.        ,  1.        ,
        -0.9       , -1.2       , -1.7       ,  1.5       , -2.1       ,
        -3.6       ,  1.2       , -1.2       ,  1.8       ,  0.        ,
        -1.6       ,  2.60830391,  2.5299999 ,  1.64      ]])

In [117]:
# Get y for model
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
y = data['labels']
label_encoder = LabelEncoder()
label_encoder.fit(y)
y_encoded = label_encoder.transform(y)
y = to_categorical(y_encoded)

In [118]:
# Kmeans Model
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10)
kmeans.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [119]:
# Random forrest model
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=20)
rf = rf.fit(X, y_encoded)

In [120]:
# SVM Model
from sklearn.svm import SVC 
svm = SVC(kernel='poly',gamma='auto',C=.25)
svm.fit(X, y_encoded)

SVC(C=0.25, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [121]:
# Neural Network model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
cnn = Sequential()
cnn.add(Dense(units=40, activation='relu', input_dim=X.shape[1]))
cnn.add(Dense(units=20, activation='relu'))
cnn.add(Dense(units=y.shape[1], activation='softmax'))
cnn.summary()

cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn.fit(X, y, epochs=5, shuffle=True, verbose=0)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 40)                1000      
_________________________________________________________________
dense_16 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_17 (Dense)             (None, 3)                 63        
Total params: 1,883
Trainable params: 1,883
Non-trainable params: 0
_________________________________________________________________


In [122]:
# Model loss and Accuray
print(f"Random Forrest - Score: {rf.score(X, y_encoded)}")
print(f"Support Vector - Score: {svm.score(X, y_encoded)}")
cnn_loss, cnn_accuracy = cnn.evaluate(X, y, verbose=0)
print(f"Normal Neural Network - Loss: {cnn_loss}, Accuracy: {cnn_accuracy}")

Random Forrest - Score: 1.0
Support Vector - Score: 1.0
Normal Neural Network - Loss: 0.01675560207334919, Accuracy: 0.9982455968856812


In [123]:
# Set Kmeans groups on testdata
data["grp"] = kmeans.labels_
data_group = data.groupby('grp').agg(['sum','count', 'min', 'max'])
buygroup = data_group['labels'][['sum']].idxmax()[0]
sellgroup = data_group['labels'][['sum']].idxmin()[0]
def setkmeans(x):
    if x == sellgroup:
        return -1
    elif x == buygroup:
        return 1
    else:
        return 0

In [124]:
# Load data and select trainging hour and test hour
testhour = 8
df = pd.read_csv('combined.csv')
cols = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
cols10 = ['31', '32', '33', '34', '35', '36', '37', '38', '39', '40']
df['begin'] = df[cols].sum(axis=1)
df['sum10'] = df[cols10].sum(axis=1)
testdata = df[(df['hour'] == testhour) & (df['month'] == 2)]
del df
testdata.head()

,Unnamed: 0,year,month,day,hour,minute,Weekday,Change,1,2,...,56,57,58,59,60,dow_open,euf_open,us_6mn,begin,sum10
28777,28777,2018,2,1,8,0,5,0.8,0.8,1.6,...,0.6,4.5,1.9,-0.2,-5.5,2.608304,2.53,1.64,-1.600000e+00,-2.3
28778,28778,2018,2,1,8,1,5,1.6,1.6,-2.3,...,4.5,1.9,-0.2,-5.5,-0.8,2.608304,2.53,1.64,-4.440892e-16,2.1
28779,28779,2018,2,1,8,2,5,-2.3,-2.3,2.4,...,1.9,-0.2,-5.5,-0.8,-0.3,2.608304,2.53,1.64,-4.000000e+00,8.0
28780,28780,2018,2,1,8,3,5,2.4,2.4,-1.6,...,-0.2,-5.5,-0.8,-0.3,-1.3,2.608304,2.53,1.64,2.600000e+00,4.7
28781,28781,2018,2,1,8,4,5,-1.6,-1.6,-2.1,...,-5.5,-0.8,-0.3,-1.3,2.1,2.608304,2.53,1.64,3.100000e+00,4.0


In [125]:
formodel = testdata.iloc[:,18:38]
formodel['begin'] = testdata['begin']
formodel['dow_open'] = testdata['dow_open']
formodel['euf_open'] = testdata['euf_open']
formodel['us_6mn'] = testdata['us_6mn']
formodel = formodel.to_numpy()
formodel.shape

(1140, 24)

In [126]:
# Pull out validation data and make predicts
testdata['grp'] = kmeans.fit_predict(formodel)
testdata["kmn"] = testdata["grp"].apply(setkmeans)
testdata['rf'] = label_encoder.inverse_transform(rf.predict(formodel))
testdata['svm'] = label_encoder.inverse_transform(svm.predict(formodel))
testdata['cnn'] = label_encoder.inverse_transform(cnn.predict_classes(formodel))
testdata['abs'] = testdata['kmn'].abs() + testdata['rf'].abs() + testdata['svm'].abs() + testdata['cnn'].abs()
testdata['tot'] = testdata['kmn'] + testdata['rf'] + testdata['svm'] + testdata['cnn']
testdata['sum10'].describe()

count    1140.000000
mean        0.239649
std         8.300460
min       -68.300000
25%        -4.300000
50%         0.600000
75%         5.500000
max        33.500000
Name: sum10, dtype: float64

In [127]:
testdata = testdata.drop(testdata[(testdata['abs'] == 0)].index)
testdata['sum10'].describe()

count    374.000000
mean      -0.412032
std        9.628891
min      -68.300000
25%       -4.700000
50%        0.800000
75%        5.400000
max       33.500000
Name: sum10, dtype: float64

In [133]:
testdata = testdata.drop(testdata[(testdata['tot'] == -1)].index)
testdata = testdata.drop(testdata[(testdata['tot'] == 0)].index)
testdata = testdata.drop(testdata[(testdata['tot'] == 1)].index)
testdata['sum10'].describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: sum10, dtype: float64

In [132]:
totalrows = testdata.shape[0]
totalcols = testdata.shape[1]
profit = []
for i in range(totalrows):      
    if (testdata.iloc[i,totalcols-1] < -1):
        temp = -testdata['sum10'].iloc[i]
    else:
        temp = testdata['sum10'].iloc[i]
    
    profit.append(temp)

print("Est Profit: " + testdata.columns[totalcols-1] + " " + str(sum(profit)))

Est Profit: tot -25.3


In [130]:
#testdata.to_csv('test_models.csv')